In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.cluster import KMeans
from fastdtw import fastdtw
from sklearn.cluster import KMeans
from fastdtw import fastdtw
from sklearn.preprocessing import StandardScaler
import glob


entorno = 'local'  # Elegir "VM" o "local" para correr en entorno local
nombre_experimento = 'clusterizacion_dtw'
ventana_input = 12
ventana_output = 2
num_clusters = 20
modo_test = True  # Configura True para usar solo 1000 primeras series en modo de prueba


# Configurar entorno
if entorno == 'VM':
    carpeta_datasets = '~/buckets/b1/datasets'
    carpeta_exp_base = '~/buckets/b1/exp'
elif entorno == 'local':
    carpeta_datasets = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos'
    carpeta_exp_base = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Resultados'
else:
    raise Exception("Entorno especificado incorrectamente")

carpeta_exp = os.path.join(carpeta_exp_base, nombre_experimento)
if not os.path.exists(carpeta_exp):
    os.makedirs(carpeta_exp)
    

In [2]:
# Cargar datos
sell_in = pd.read_csv(os.path.join(carpeta_datasets, 'sell-in.txt'), delimiter='\t')
tb_productos = pd.read_csv(os.path.join(carpeta_datasets, 'tb_productos_descripcion.txt'), delimiter='\t')
tb_stocks = pd.read_csv(os.path.join(carpeta_datasets, 'tb_stocks.txt'), delimiter='\t')
productos_predecir = pd.read_csv(os.path.join(carpeta_datasets, 'productos_a_predecir.txt'), delimiter='\t')


In [3]:
# Preparar ventas por producto, cliente, mes
ventas_producto_mes = sell_in.groupby(['periodo', 'product_id', 'customer_id'])['tn'].sum().reset_index()
ventas_producto_mes['Timestamp'] = pd.to_datetime(ventas_producto_mes['periodo'], format='%Y%m')
ventas_producto_mes.drop(columns=['periodo'], inplace=True)


#Filtrar el dataframe por los productos sobre los cuales se realizarán predicciones 
lista_productos_predecir = list(productos_predecir['product_id'])
ventas_producto_mes = ventas_producto_mes[ventas_producto_mes['product_id'].isin(lista_productos_predecir)]

In [4]:
if not os.path.isfile(os.path.join(carpeta_exp, 'tamaño_series.csv')):
    #Analizar tamaño de las series producto-cliente
    tamaño_series = pd.DataFrame()
    cantidad_productos= len(ventas_producto_mes['product_id'].unique())


    j = 1
    for producto in ventas_producto_mes['product_id'].unique():
        clientes_producto = ventas_producto_mes[ventas_producto_mes['product_id'] == producto]
        cantidad_clientes = len(clientes_producto['customer_id'].unique())
        i= 1
        for cliente in clientes_producto['customer_id'].unique():
            print(f'Procesando cliente {i} de {cantidad_clientes} - Producto {j} de {cantidad_productos}',  end="\r")
            serie_producto_cliente = clientes_producto[clientes_producto['customer_id'] == cliente]
            tamaño_series = pd.concat([tamaño_series, pd.DataFrame({
                'product_id': [producto],
                'customer_id': [cliente],
                'cantidad_datos': [len(serie_producto_cliente)]
            })])
            i+=1
        j += 1
    
    tamaño_series.to_csv(os.path.join(carpeta_exp, 'tamaño_series.csv'), index = False)
else:
    tamaño_series = pd.read_csv(os.path.join(carpeta_exp, 'tamaño_series.csv'))

In [5]:
# Ordenar por cantidad de datos en orden descendente
tamaño_series = tamaño_series.sort_values(by='cantidad_datos', ascending=False)

N= 400
# Tomar las series con datos completos y hacer un sampleo de N series
series_completas = tamaño_series[tamaño_series['cantidad_datos'] == 36]
sample_series_completas = series_completas.sample(n=N, random_state=42)  

# Hacer un sampleo de N series del resto, evitando las que tienen menos de 12 meses de historia
resto = tamaño_series.iloc[len(series_completas):]
resto = resto[resto['cantidad_datos'] >= 12]
sample_N = resto.sample(n=N, random_state=42) 

# Combinar los dos DataFrames
series_sampled = pd.concat([sample_series_completas, sample_N], ignore_index=True)

# Mostrar el DataFrame resultado
print(series_sampled)

     product_id  customer_id  cantidad_datos
0         20140        10011              36
1         20076        10008              36
2         20329        10009              36
3         20177        10005              36
4         20009        10005              36
..          ...          ...             ...
795       20864        10350              26
796       20288        10012              28
797       20745        10042              13
798       20599        10031              12
799       20642        10105              15

[800 rows x 3 columns]


In [6]:
# Filtrar la serie de acuerdo a los resultados
ventas_producto_mes_sampling = pd.DataFrame(columns=ventas_producto_mes.columns)

fecha_inicio = min(ventas_producto_mes['Timestamp'])
fecha_fin = max(ventas_producto_mes['Timestamp'])

date_range = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='MS')

# Iterar sobre los pares de product_id y customer_id en series_sampled
for index, row in series_sampled.iterrows():
    product_id = row['product_id']
    customer_id = row['customer_id']

    # Filtrar serie ventas_producto_mes por el par product_id y customer_id
    filtrado = ventas_producto_mes[(ventas_producto_mes['product_id'] == product_id) & (ventas_producto_mes['customer_id'] == customer_id)].copy()
    #Rellenar serie con 0 donde no hay compra
    filtrado.set_index('Timestamp', inplace = True)
    filtrado = filtrado.reindex(date_range)
    filtrado.index.name = 'Timestamp'
    filtrado['tn'] = filtrado['tn'].fillna(0)
    filtrado['product_id'] = int(product_id)
    filtrado['customer_id'] = int(customer_id)
    filtrado = filtrado.reset_index()
    scaler = StandardScaler()
    filtrado['tn'] = scaler.fit_transform(np.array(filtrado['tn']).reshape(-1, 1))
    
    # Concatenar los resultados filtrados a la serie_concatenada
    ventas_producto_mes_sampling = pd.concat([ventas_producto_mes_sampling, filtrado])
    
# Mostrar el resultado final
print(ventas_producto_mes_sampling)

   product_id customer_id        tn  Timestamp
0       20140       10011 -0.926628 2017-01-01
1       20140       10011 -1.693996 2017-02-01
2       20140       10011  0.673883 2017-03-01
3       20140       10011  2.044184 2017-04-01
4       20140       10011 -2.000943 2017-05-01
..        ...         ...       ...        ...
31      20642       10105 -0.664551 2019-08-01
32      20642       10105  0.747917 2019-09-01
33      20642       10105 -0.664551 2019-10-01
34      20642       10105 -0.664551 2019-11-01
35      20642       10105  0.182601 2019-12-01

[28800 rows x 4 columns]


In [7]:
# Calcular la matriz de distancias DTW
productos_clientes = ventas_producto_mes_sampling.groupby(['product_id', 'customer_id'])['tn'].apply(list).reset_index()

series = productos_clientes['tn'].tolist()
distancias = np.zeros((len(series), len(series)))

for i in range(len(series)):
    for j in range(i + 1, len(series)):
        print(f'Procesando fila {i}, columna {j} de matriz de {len(series)}*{len(series)}',  end="\r")
        distancia, _ = fastdtw(series[i], series[j])
        distancias[i, j] = distancia
        distancias[j, i] = distancia

In [11]:
# Clusterizar las series de tiempo
num_clusters = 25
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
productos_clientes['cluster'] = kmeans.fit_predict(distancias)

C:\Users\alope\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [12]:
# Guardar las series con su cluster asociado
productos_clientes.to_csv(os.path.join(carpeta_exp, 'series_con_clusters.csv'), index=False)

In [51]:
import matplotlib.pyplot as plt

#Cargar serie de clusters
productos_clientes = pd.read_csv(os.path.join(carpeta_exp, 'series_con_clusters.csv'))
num_clusters = len(productos_clientes['cluster'].unique())

# Generar gráficos de las series por cluster
# Borrar gráficos existentes de la carpeta
patron = 'cluster_*.png'
archivos_a_eliminar = glob.glob(os.path.join(carpeta_exp, patron))
for archivo in archivos_a_eliminar:
    os.remove(archivo)

ventas_producto_mes_copia = ventas_producto_mes.copy()

#Graficar clusters
for cluster in range(num_clusters):
    series_cluster = productos_clientes[productos_clientes['cluster'] == cluster]
    plt.figure(figsize=(12, 8))

    for _, row in series_cluster.iterrows():
        product_id = row['product_id']
        customer_id = row['customer_id']
        serie = ventas_producto_mes_copia[(ventas_producto_mes_copia['product_id'] == product_id) &
                                    (ventas_producto_mes_copia['customer_id'] == customer_id)]
        serie_tn = serie['tn']
        scaler = StandardScaler()
        
        serie_tn_escalada = scaler.fit_transform(np.array(serie_tn).reshape(-1,1))
        ventas_producto_mes_copia.loc[serie.index, 'tn'] = serie_tn_escalada
        
        serie_plot = ventas_producto_mes_copia[(ventas_producto_mes_copia['product_id'] == product_id) &
                                    (ventas_producto_mes_copia['customer_id'] == customer_id)]['tn']
        serie_plot.plot(label=f'Producto: {product_id}, Cliente: {customer_id}', alpha=0.5)

    plt.title(f'Cluster {cluster}')
    plt.xlabel('Fecha')
    plt.ylabel('Ventas')
    plt.legend()
    plt.savefig(os.path.join(carpeta_exp, f'cluster_{cluster}.png'))
    plt.close()

In [44]:
serie_escalada

Index([  63150,  144971,  233429,  319858,  412203,  508407,  586267,  676271,
        768185,  860609,  953316, 1035071, 1115134, 1201255, 1298718, 1384361,
       1468759, 1557906, 1637572, 1723422, 1799763, 1878476, 1952393, 2022134,
       2091138, 2171180, 2258395, 2339511, 2411719, 2491596, 2573565, 2642903,
       2721084, 2798583, 2872843, 2933726],
      dtype='int64')